# This is an Notebook which can evalute multiple files.  It implements a number of features:

- TopK
- Removing recent referrals
- Thresholding 
- Tables and visualization
- Config files for standardized processes. 



In [41]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path

#path append
sys.path.append(os.path.join(Path.cwd(), 'modules'))
import Evaluate, Helper, Present, Score, Synthetic


<H3> Imports</H3>

### File configuration
<p> Just edit the configuration file path/address. prediction_files contain configs for different prediction files. config_file contains other configs for referral, visualization, experiment etc. </p>

In [42]:
config_file = 'config/config.yaml'
prediction_config_files = ['config/predictions/pred1.yaml', 'config/predictions/pred2.yaml']
generate_data=True


### Load Configuration and Referrals

In [43]:
'''
Transferred referral data loading inside the c_p for loop below, because referral_data is being changed for date format adjustment. 
As referral_data.copy() is not allowed for memory issues. The overhead shouldn't be much.
'''

#Load Configuration
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

#Generate Data (if required)
if generate_data:
    Synthetic.generate_synthetic_event_data(c_gen)
    Synthetic.generate_synthetic_prediction_data(c_gen)

### Evaluate Models
For each configuration file provided it will evaluate the models. 

In [44]:
for c_p in predictions:    
    prediction=Helper.read_file(directory=c_p['dir'],file=c_p['file'],file_format=c_p['file_format'],aws=c_p['aws'],bucket= c_p['bucket'])
    referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
    
    #TBD Implement a check that there is a match between prediction and evaluation file.
    
    all_model_evaluations = Evaluate.evaluate(c_p, c_e, c_r, referrals, prediction) 
    Present.present_evaluation(c_p, c_r, c_e, c_visual, all_model_evaluations)

Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Columns from dataframe (referral/prediction): ['PERSON_ID', 'MYR', 'lin_reg', 'rand_forest', 'xg_boost', 'sgmm']
Columns from config(c_r/c_p): ['PERSON_ID', 'MYR']
Number of Unique IDs in Referral: 25
Number of Unique IDs in Prediction: 500
Number of Intersected IDs: 15
Precision


{'lin_reg_window_[0,3]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,3]_top_k_@k=60': 0.016666666666666666,
 'lin_reg_window_[0,3]_thresholding_@p>=0.5': 0.0038910505836575876,
 'lin_reg_window_[0,3]_thresholding_@p>=0.6': 0.005154639175257732,
 'lin_reg_window_[0,6]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,6]_top_k_@k=60': 0.016666666666666666,
 'lin_reg_window_[0,6]_thresholding_@p>=0.5': 0.0038910505836575876,
 'lin_reg_window_[0,6]_thresholding_@p>=0.6': 0.005154639175257732,
 'lin_reg_window_[0,12]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,12]_top_k_@k=60': 0.016666666666666666,
 'lin_reg_window_[0,12]_thresholding_@p>=0.5': 0.007782101167315175,
 'lin_reg_window_[0,12]_thresholding_@p>=0.6': 0.010309278350515464}

Recall


{'lin_reg_window_[0,3]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,3]_top_k_@k=60': 0.25,
 'lin_reg_window_[0,3]_thresholding_@p>=0.5': 0.25,
 'lin_reg_window_[0,3]_thresholding_@p>=0.6': 0.25,
 'lin_reg_window_[0,6]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,6]_top_k_@k=60': 0.25,
 'lin_reg_window_[0,6]_thresholding_@p>=0.5': 0.25,
 'lin_reg_window_[0,6]_thresholding_@p>=0.6': 0.25,
 'lin_reg_window_[0,12]_top_k_@k=50': 0.0,
 'lin_reg_window_[0,12]_top_k_@k=60': 0.14285714285714285,
 'lin_reg_window_[0,12]_thresholding_@p>=0.5': 0.2857142857142857,
 'lin_reg_window_[0,12]_thresholding_@p>=0.6': 0.2857142857142857}

F1_Score


...

{'sgmm_window_[0,3]_top_k_@k=50': 0.9541151044339882,
 'sgmm_window_[0,3]_top_k_@k=60': 0.9541151044339882,
 'sgmm_window_[0,3]_thresholding_@p>=0.5': 0.9541151044339882,
 'sgmm_window_[0,3]_thresholding_@p>=0.6': 0.9541151044339882,
 'sgmm_window_[0,6]_top_k_@k=50': 0.9541151044339882,
 'sgmm_window_[0,6]_top_k_@k=60': 0.9541151044339882,
 'sgmm_window_[0,6]_thresholding_@p>=0.5': 0.9541151044339882,
 'sgmm_window_[0,6]_thresholding_@p>=0.6': 0.9541151044339882,
 'sgmm_window_[0,12]_top_k_@k=50': 0.9565160929108135,
 'sgmm_window_[0,12]_top_k_@k=60': 0.9565160929108135,
 'sgmm_window_[0,12]_thresholding_@p>=0.5': 0.9565160929108135,
 'sgmm_window_[0,12]_thresholding_@p>=0.6': 0.9565160929108135}

ROC Score


{'sgmm_window_[0,3]_top_k_@k=50': 0.5835030549898167,
 'sgmm_window_[0,3]_top_k_@k=60': 0.5835030549898167,
 'sgmm_window_[0,3]_thresholding_@p>=0.5': 0.5835030549898167,
 'sgmm_window_[0,3]_thresholding_@p>=0.6': 0.5835030549898167,
 'sgmm_window_[0,6]_top_k_@k=50': 0.5835030549898167,
 'sgmm_window_[0,6]_top_k_@k=60': 0.5835030549898167,
 'sgmm_window_[0,6]_thresholding_@p>=0.5': 0.5835030549898167,
 'sgmm_window_[0,6]_thresholding_@p>=0.6': 0.5835030549898167,
 'sgmm_window_[0,12]_top_k_@k=50': 0.5181498829039812,
 'sgmm_window_[0,12]_top_k_@k=60': 0.5181498829039812,
 'sgmm_window_[0,12]_thresholding_@p>=0.5': 0.5181498829039812,
 'sgmm_window_[0,12]_thresholding_@p>=0.6': 0.5181498829039812}

Brier Score Loss


{'sgmm_window_[0,3]_top_k_@k=50': 0.3191272124830303,
 'sgmm_window_[0,3]_top_k_@k=60': 0.3191272124830303,
 'sgmm_window_[0,3]_thresholding_@p>=0.5': 0.3191272124830303,
 'sgmm_window_[0,3]_thresholding_@p>=0.6': 0.3191272124830303,
 'sgmm_window_[0,6]_top_k_@k=50': 0.3191272124830303,
 'sgmm_window_[0,6]_top_k_@k=60': 0.3191272124830303,
 'sgmm_window_[0,6]_thresholding_@p>=0.5': 0.3191272124830303,
 'sgmm_window_[0,6]_thresholding_@p>=0.6': 0.3191272124830303,
 'sgmm_window_[0,12]_top_k_@k=50': 0.3200993740991919,
 'sgmm_window_[0,12]_top_k_@k=60': 0.3200993740991919,
 'sgmm_window_[0,12]_thresholding_@p>=0.5': 0.3200993740991919,
 'sgmm_window_[0,12]_thresholding_@p>=0.6': 0.3200993740991919}